In [1]:
import boto3
from botocore.client import Config
import os

In [2]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

# List objects in a specific bucket
objects = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in objects:
    print("Objects found in bucket")
else:
    print("  No objects found.")

Objects found in bucket


In [3]:
project_name_param = os.getenv('project_name')
experiment_name_param = os.getenv('experiment_name')
run_name_param = os.getenv('run_name')

In [8]:
current_directory = os.getcwd()
newmodel_folder = "newmodal"
newmodel_path = os.path.join(current_directory, newmodel_folder)

if not os.path.exists(newmodel_path):
    os.makedirs(newmodel_path)

model_to_score = f"{project_name_param}/{experiment_name_param}/Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx"
download_path_model_to_score = os.path.join(newmodel_path, f"Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx")

try:
    s3_client.download_file(bucket_name, model_to_score, download_path_model_to_score)
    print(f"Object {model_to_score} downloaded successfully to '{download_path_model_to_score}'")
except Exception as e:
    print(f"Error occurred: {e}")

Object fracture/development/Classif-Exp-development-Run-5-epochs.onnx downloaded successfully to '/opt/app-root/src/Beroepsproduct/Models/Scoring/newmodal/Classif-Exp-development-Run-5-epochs.onnx'


In [9]:
download_folder = f"{project_name_param}/production/"
production_folder = "production"
production_path = os.path.join(current_directory, production_folder)

if not os.path.exists(production_path):
    os.makedirs(production_path)

try:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=download_folder)

    if 'Contents' in response:
        for obj in response['Contents']:
            object_name = obj['Key']
            base_name = os.path.basename(object_name)
            download_path = os.path.join(production_path, base_name)

            s3_client.download_file(bucket_name, object_name, download_path)
            print(f"Object '{object_name}' downloaded successfully as '{download_path}'")
    else:
        print(f"No objects found in the folder '{download_folder}'")

except Exception as e:
    print(f"Error occurred: {e}")

Object 'fracture/production/Classif-Exp-development-Run-3-epochs.onnx' downloaded successfully as '/opt/app-root/src/Beroepsproduct/Models/Scoring/production/Classif-Exp-development-Run-3-epochs.onnx'


In [10]:
import os
import zipfile

folders_to_zip = ['newmodal', 'production']
zip_filename = 'modals.zip'

current_directory = os.getcwd()
zip_path = os.path.join(current_directory, zip_filename)

def add_directory_to_zip(zipf, folder, folder_base):
    for root, dirs, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, folder_base))

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for folder in folders_to_zip:
        folder_path = os.path.join(current_directory, folder)
        if os.path.exists(folder_path):
            add_directory_to_zip(zipf, folder_path, current_directory)
        else:
            print(f"Folder '{folder}' does not exist and will be skipped.")

print(f"Folders '{', '.join(folders_to_zip)}' have been zipped into '{zip_path}'.")

Folders 'newmodal, production' have been zipped into '/opt/app-root/src/Beroepsproduct/Models/Scoring/modals.zip'.
